# Taller re:Invent 2019 de Amazon Personalize

![Esquema principal](static/imgs/image.png)

## Programa

A grandes rasgos, el trabajo con Amazon Personalize sigue los pasos del siguiente diagrama:

![Diagrama de flujo](static/imgs/personalize_process.png)

A continuación, verá el proceso específico que se seguirá:

1. Importaciones y configuración
1. Preparación de datos
1. Importación de datos
1. Selección de una receta
1. Entrenamiento de una solución
1. Implementación de una campaña
1. Obtención de recomendaciones
1. Interacciones en tiempo real
1. Conclusión
1. Bonus: Exportación masiva de recomendaciones

En cada uno de estos pasos verá y ejecutará fragmentos de código escritos en Python por medio de nuestro SDK de Boto3, estos fragmentos pueden modificarse para ser componentes de su integración de producción con Personalize. 

Este cuaderno lo guiará a través de los pasos para crear un modelo de recomendación de películas que se adapte a usuarios específicos. El objetivo es recomendar películas basadas en el historial de interacciones positivas de esos usuarios con las películas.

Los datos se facilitan a través del [proyecto MovieLens](https://movielens.org), sobre el que podrá leer más adelante si está interesado.

El contenido que figura a continuación se ha redactado para guiarlo a través del proceso dentro del plazo de una sesión del taller. Puede implementar la plantilla de CloudFormation que encontró en GitHub más tarde en su propia cuenta y puede recorrer los cuadernos 1 a 3 para volver a hacer el mismo ejercicio. Además, puede actualizar el contenido de las celdas para reflejar sus propios datos y es un camino eficaz para crear un modelo de recomendación personalizado para su caso de uso.

## Cómo utilizar el cuaderno

El código se divide en celdas como la siguiente. En la parte superior de esta página, puede hacer clic en un botón triangular `Run` para ejecutar cada celda y pasar a la siguiente, o puede presionar `Shift` + `Enter` o `Shift` + `Return`(para usuarios de Mac) mientras está en la celda para ejecutarla y pasar a la siguiente.


A medida que una celda se ejecuta, verá una línea al costado que muestra un `*` mientras la celda se está ejecutando o se actualizará a un número para indicar la última celda que terminó de ejecutarse luego de que haya terminado de ejecutar todo el código dentro de una celda.


Solo debe seguir las instrucciones que aparecen a continuación y ejecutar las celdas para comenzar a utilizar Amazon Personalize.



## Importaciones y configuración 

Python viene con una amplia colección de bibliotecas y necesitamos importarlas, así como las que se instalan para ayudarnos, como boto3 (el AWS SDK) y Pandas/Numpy, que son herramientas básicas de la ciencia de datos. La celda a continuación los importará para utilizarlos aquí. También actualizará la biblioteca boto3 a la última versión. Es posible que vea una advertencia (texto amarillo) o un error (texto rojo), esto está perfectamente bien, simplemente continúe ejecutando la siguiente celda.

In [ ]:
# Update boto3
!pip install --upgrade boto3
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import datetime
import uuid

A continuación, querrá validar que su entorno puede comunicarse correctamente con Amazon Personalize. Las líneas que figuran a continuación hacen precisamente eso.

In [ ]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client(service_name='personalize-events')

La última parte de la configuración consiste en determinar la región en la que va a realizar este taller. La celda a continuación lo hará y lo asignará a la variable `region`.

In [ ]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]

## Preparación de datos

Para empezar, necesitará una colección de puntos de datos en los que los usuarios hayan interactuado con el contenido de alguna manera. Amazon Personalize asume que si se registra una interacción es positiva. Más adelante verá cómo lo utilizamos. 

La celda a continuación descargará los datos que necesita, extraerá el contenido de un archivo Zip y luego mostrará una pequeña parte en su pantalla.

In [ ]:
!wget -N http://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -o ml-100k.zip
data = pd.read_csv('./ml-100k/u.data', sep='\t', names=['USER_ID', 'ITEM_ID', 'RATING', 'TIMESTAMP'], engine='python')
pd.set_option('display.max_rows', 5)
data

Como puede ver, los datos contienen un UserID, un ItemID, una clasificación y una marca temporal.

No necesitamos la columna `Rating`, por lo que se eliminará antes de guardar el archivo.

Una vez preparados los datos, los guardará en un archivo CSV localmente con la última línea.

In [ ]:
data = data[['USER_ID', 'ITEM_ID', 'TIMESTAMP']] # select columns that match the columns in the schema below
filename = "movie-lens-ml-100k-prepared.csv"
data.to_csv(filename, index=False)

Antes de cargar los datos a S3 tendrá que crear un bucket, la celda de abajo lo hará.

In [ ]:
print(region)
s3 = boto3.client('s3')
account_id = boto3.client('sts').get_caller_identity().get('Account')
bucket_name = account_id + "reinventpersonalizeworkshop"
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

Ahora puede cargar el archivo a S3 y estará listo para importarlo a Amazon Personalize.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(filename).upload_file(filename)

## Importación de datos

Los pasos para importar sus datos son los siguientes:

1. Cree un grupo de conjuntos de datos.
1. Determine el esquema de su conjunto de datos.
1. Cree un conjunto de datos con su esquema.
1. Ejecute un ImportJob para cargar los datos que se utilizarán con Personalize.


El grupo de conjuntos de datos de Amazon Personalize es la forma en que su información se aísla de cualquier otro experimento. No se comparte ningún tipo de información entre estos grupos. Un grupo de conjuntos de datos puede contener sus datos de interacción, metadatos de elementos, metadatos de usuarios, rastreadores de eventos, soluciones y campañas. Para obtener más información sobre ellos, diríjase a: https://docs.aws.amazon.com/personalize/latest/dg/API_DatasetGroup.html 

Comience por crear un grupo de conjuntos de datos

#### Crear grupo de conjuntos de datos

In [ ]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-RI-demo"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

#### Grupo de conjuntos de datos. Crear bucle

La creación de un grupo de conjuntos datos tarda unos segundos, por lo que la celda de abajo sondeará hasta que el grupo de datos esté activo y pueda continuar con el taller. Una vez que diga `ACTIVE`, continúe con el paso `Create Schema`.


In [ ]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(5)

#### Crear un esquema


Ahora puede determinar el esquema de su conjunto de datos. En este taller el tiempo es limitado, por lo que solo se utilizarán los datos de interacción del usuario o datos de interacción para abreviar. La celda que aparece a continuación contiene los elementos necesarios para este conjunto de datos y se corresponde con la estructura del CSV que cargó anteriormente.

In [ ]:
schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

La celda a continuación creará un esquema dentro de Amazon Personalize que se puede conectar a un conjunto de datos. Así es como Personalize entiende el contenido dentro de su archivo CSV.

In [ ]:
create_schema_response = personalize.create_schema(
    name = "personalize-ri-demo-schema",
    schema = json.dumps(schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

#### Crear conjuntos de datos


A continuación, cree el conjunto de datos para las interacciones y asígnele el esquema proporcionado así como al grupo de conjuntos de datos que creó anteriormente con este código:

In [ ]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ri-interactions",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

#### Adjuntar la política al bucket de S3

Antes de que pueda ejecutar el trabajo de importación de sus datos deberá adjuntar una política de bucket a su bucket de S3 para permitir que Personalize se comunique con él, así como un rol de IAM para que el servicio lo utilice dentro de esta cuenta de AWS.

In [ ]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

Tenga en cuenta que los cambios de IAM en la celda a continuación tardan solo unos segundos en ser confirmados, por lo que esta sección tardará 1 minuto en completarse. Ejecute la celda de abajo a continuación.

#### Crear un rol de Personalize

In [ ]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRIDemo"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

Finalmente está listo para importar los datos a Personalize. La primera celda a continuación iniciará el proceso y la segunda contiene un bucle while que realizará un reconocimiento del servicio para determinar cuándo se ha completado el trabajo de importación. 

Durante este taller está importando un conjunto de datos relativamente pequeño y puede que le parezca que esto lleva un tiempo para un archivo tan pequeño. El principal uso del tiempo aquí es el aprovisionamiento de recursos dedicados que realmente ejecutarán la tarea. Esto permite, por ejemplo, que Personalize proporcione el cumplimiento de la HIPAA, también significa que solo porque sus archivos son más grandes no va a tardar mucho tiempo, una vez que los recursos están en funcionamiento la importación es bastante rápida.

El proceso de importación puede tardar hasta 20 minutos en completarse. 

#### Crear un trabajo de importación de conjuntos de datos

In [ ]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ri-import",
    datasetArn = dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

#### Bucle de importación de conjuntos de datos

La celda que se muestra a continuación tardará un poco más en completarse. Se trata de un control para saber cuándo su conjunto de datos se ha importado completamente en Personalize. El proceso debería tardar unos 20 minutos en completarse. La mayor parte del tiempo se dedica a aprovisionar la infraestructura de forma oculta. Esto significa que, en la mayoría de los casos, los archivos más grandes no tardan mucho en importarse.

Ejecute la celda a continuación, luego avance llegue a un estado `ACTIVE`.


In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

## Selección de una receta

Los algoritmos disponibles dentro de Personalize se denominan recetas, el código siguiente le dará una lista completa de ellas. Cada uno de ellos tiene varios casos de uso y se puede encontrar información detallada aquí: https://docs.aws.amazon.com/personalize/latest/dg/working-with-predefined-recipes.html

En este taller, utilizará HRNN (Red Neuronal Recurrente Jerárquica) citando los documentos:

```
La HRRN es una red neuronal recurrente jerárquica, que puede modelar las interacciones entre el usuario y los elementos a lo largo de un marco temporal determinado. Utilice la receta HRNN cuando el comportamiento del usuario cambie con el tiempo, lo que se conoce como el problema de la intención evolutiva.

Para entrenar un modelo, HRNN utiliza el conjunto de datos de interacciones de un grupo de datos. Un grupo de conjuntos de datos es un bloque de conjuntos de datos relacionados, que puede incluir los conjuntos de datos usuarios, elementos e interacciones.
```

A continuación, le presentamos un documento que lo explica más detalladamente en relación con las recomendaciones: https://openreview.net/pdf?id=ByzxsrrkJ4 


Aquí, se utiliza como ejemplo de un sistema de recomendación creado mediante redes neuronales profundas y que se puede entrenar únicamente mediante nuestros datos de interacciones. Este es un buen punto de partida para realizar sus propios experimentos más adelante.

In [ ]:
personalize.list_recipes()

La celda a continuación selecciona la receta `HRNN`.

In [ ]:
recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"

## Entrenamiento de una solución

Un modelo entrenado con los datos del cliente se denomina solución dentro de Amazon Personalize, y se trata de versiones, por lo que primero se crea una solución y luego una versión de la misma. Las versiones se utilizan para hacer un seguimiento de la mejora del modelo a lo largo del tiempo basándose en los nuevos datos disponibles. 

La creación de una solución en sí misma es casi instantánea, pero el entrenamiento real para crear una versión puede llevar un poco de tiempo. Este suele ser el periodo de espera más largo del proceso. Si lo hace fuera de un taller, es un buen momento para revisar sus correos electrónicos, tomar un café, etc.

#### Crear una solución

In [ ]:
create_solution_response = personalize.create_solution(
    name = "personalize-ri-soln-hrnn",
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn
)

solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

#### Crear una versión de la solución

In [ ]:
create_solution_version_response = personalize.create_solution_version(
    solutionArn = solution_arn
)

solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

#### Crear un bucle de la versión de la solución

El entrenamiento de un modelo puede llevar un poco de tiempo, ya que suele tardar al menos 20 minutos en completarse. Ejecute la celda a continuación y espere de nuevo a que llegue a un estado `ACTIVE` antes de pasar a la siguiente parte.

In [ ]:
current_time = datetime.datetime.now()
print("Training Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Training Completed on: ", current_time.strftime("%I:%M:%S %p"))

#### Obtener las métricas de la versión de la solución

Una vez que disponga de su solución y de su versión, puede obtener las métricas para juzgar su rendimiento.

In [ ]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)

print(json.dumps(get_solution_metrics_response, indent=2))

## Crear y esperar una campaña

Ahora que tiene una versión de la solución que funciona, tendrá que crear una campaña para utilizarla con sus aplicaciones. Una campaña es simplemente una copia alojada de su modelo. Una vez más, habrá una breve espera, así que después de la ejecución puede tomar un rápido descanso mientras se aprovisiona la infraestructura.

#### Crear una campaña

In [ ]:
create_campaign_response = personalize.create_campaign(
    name = "personalize-ri-camp",
    solutionVersionArn = solution_version_arn,
    minProvisionedTPS = 1
)

campaign_arn = create_campaign_response['campaignArn']
print(json.dumps(create_campaign_response, indent=2))

#### Esperar a que la campaña tenga un estado ACTIVO

#### Crear un bucle de la campaña

En esta sección, Personalize está implementando su modelo y eso toma unos minutos para completarse. Ejecute la celda a continuación y revise de nuevo hasta que la tarea se haya completado. Debería tardar entre 10 y 15 minutos. Una vez que la celda haya alcanzado un estado `ACTIVE`, continúe.

In [ ]:
current_time = datetime.datetime.now()
print("Deploying Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Deploying Completed on: ", current_time.strftime("%I:%M:%S %p"))

## Obtener muestras de recomendaciones

Cuando la campaña esté activa, estará lista para recibir recomendaciones. Primero tenemos que seleccionar un usuario de la colección al azar. A continuación, crearemos algunas funciones de ayuda para obtener la información de la película que se mostrará para las recomendaciones en vez de mostrar únicamente los ID.

In [ ]:
# Getting a random user:
user_id, item_id, _ = data.sample().values[0]
print("USER: {}".format(user_id))

In [ ]:
items = pd.read_csv('./ml-100k/u.item', sep='|', usecols=[0,1], encoding='latin-1', names=['ITEM_ID', 'TITLE'], index_col='ITEM_ID')

def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)-1
    return items.iloc[movie_id]['TITLE']

#### Llamar a GetRecommendations

El código que se muestra a continuación obtendrá recomendaciones para el usuario seleccionado al azar, luego coloca las recomendaciones en un marco de datos y lo renderiza.

In [ ]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_list.append(title)
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['Original Recommendations'])
recommendations_df

## Creación de un rastreador de eventos

Antes de que su sistema de recomendación pueda responder a eventos en tiempo real, necesitará un rastreador de eventos. El código que se muestra a continuación generará uno y se podrá utilizar para continuar con este laboratorio. Puede colocarle el nombre que desee.

In [ ]:
response = personalize.create_event_tracker(
    name='MovieClickTrackerRI',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']

In [ ]:
event_tracker_arn = response['eventTrackerArn']

## Simulación del comportamiento del usuario

Las líneas que se muestran a continuación proporcionan un código de ejemplo que simula la interacción de un usuario con un elemento en particular. Por lo tanto, recibirá recomendaciones distintas a las que tenía en el comienzo.

In [ ]:
session_dict = {}

In [ ]:
def send_movie_click(USER_ID, ITEM_ID):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[USER_ID]
    except:
        session_dict[USER_ID] = str(uuid.uuid1())
        session_ID = session_dict[USER_ID]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= USER_ID,
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': 'EVENT_TYPE',
        'properties': event_json
        }]
    )

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the title of the movie for the header of the column
    movie_title_clicked = get_movie_title(movie_to_click)
    # Interact with the movie
    send_movie_click(USER_ID=str(user_id), ITEM_ID=movie_to_click)
    # Sleep for 2 seconds
    time.sleep(2)
    # Get new recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        title = get_movie_title(item['itemId'])
        recommendation_list.append(title)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_title_clicked])
    # Add this dataframe to the old one
    recommendations_df = recommendations_df.join(new_rec_DF)
    return recommendations_df

Las 3 celdas siguientes simularán la adición de una película al conjunto de datos y la representación de los resultados después de cada interacción. La primera columna son las recomendaciones originales, las columnas siguientes son para cada película con la que el usuario ya ha interactuado. El encabezado de la columna es el nombre de la película con la que el usuario ha interactuado, y a continuación puede ver cómo se modifican las recomendaciones.

In [ ]:
movie_to_click = 180
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 210
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

In [ ]:
movie_to_click = 415
recommendations_df = get_new_recommendations_df(recommendations_df, movie_to_click)
recommendations_df

## Conclusión

En este taller, pudo crear con éxito un grupo de datos y un conjunto de datos basado en las interacciones, entrenó un modelo de recomendación basado en los datos, implementó una campaña para generar recomendaciones, evaluó las recomendaciones iniciales, aprovechó el seguimiento de eventos en tiempo real para mejorar aún más las recomendaciones y, a continuación, se ofrece material adicional para mostrar cómo exportar las recomendaciones en  

Este contenido permanecerá público en GitHub y se podrá utilizar dentro de su organización para crear modelos de recomendación personalizados. Consulte los otros cuadernos en el futuro.

Gracias y buena suerte en su próximo proyecto de machine learning.

MATERIAL ADICIONAL A CONTINUACIÓN:

## Recomendaciones por lote

Hasta el momento ha visto cómo generar recomendaciones a través de una llamada a la API e interactuar con rastreadores de eventos. Esto funciona bien para muchas aplicaciones, pero es posible que desee almacenar en caché todas las recomendaciones para los usuarios a nivel local o incluso estudiar las recomendaciones para obtener nuevas ideas. Para ello, Amazon Personalize también admite la exportación por lotes de sus recomendaciones a un archivo. Las celdas que se muestran a continuación lo guiarán en el envío de recomendaciones a un archivo en S3 y luego mostrarán su contenido. Puede descargar el archivo desde S3 a su computadora local también.

Primero, tendrá que crear un archivo JSON de las identificaciones de usuario.

In [ ]:
user_IDs = ['561', '233', '579']

json_input_filename = "json_input.json"
with open(json_input_filename, 'w') as json_input:
    for user_id in user_IDs:
        json_input.write('{"userId": "' + user_id + '"}\n')

Ahora cargue este archivo a S3:

In [ ]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

Una vez que el archivo de entrada está disponible en S3, debe que definir dónde irá la salida y crear un trabajo de inferencia por lotes.

In [ ]:
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

La celda a continuación crea el trabajo por lotes.

In [ ]:
personalize_rec = boto3.client(service_name='personalize')
batchInferenceJobArn = personalize_rec.create_batch_inference_job (
    solutionVersionArn = solution_version_arn,
    jobName = "RI-Workshop-Batch-Inference-Job",
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

La siguiente celda realizará un seguimiento hasta que la exportación se haya completado. ¡Este es el último bucle de espera del taller! Esto volverá a realizar un seguimiento hasta que llegue a un estado `ACTIVE`. Puede continuar una vez que se lo alcance.

In [ ]:
current_time = datetime.datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize_rec.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Cuando los datos se hayan exportado con éxito, tome el archivo y analícelo.

In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the movies
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

En la celda anterior, puede ver las diferentes recomendaciones para los usuarios proporcionados. En un escenario real, la lista de usuarios podría ser toda su base de usuarios, lo cual le permite referenciar y comparar rápidamente los resultados entre ellos.

## Bonus adicional: Limpiar

Las celdas que aparecen a continuación son totalmente opcionales si utiliza Event Engine, ya que su cuenta se borra después del taller. Si desea utilizar esto en su propia cuenta más adelante, las celdas de abajo borrarán los recursos creados para evitar cargos continuos.

In [ ]:
# Delete the campaign:
personalize.delete_campaign(campaignArn=campaign_arn)
time.sleep(60)

In [ ]:
# Delete the solution
personalize.delete_solution(solutionArn=solution_arn)
time.sleep(60)

In [ ]:
# Delete the event tracker
personalize.delete_event_tracker(eventTrackerArn=event_tracker_arn)
time.sleep(60)

In [ ]:
# Delete the interaction dataset
personalize.delete_dataset(datasetArn=dataset_arn)
time.sleep(60)

In [ ]:
# Delete the event dataset
event_interactions_dataset_arn = dataset_arn
event_interactions_dataset_arn = event_interactions_dataset_arn.replace("INTERACTIONS", "EVENT_INTERACTIONS")
personalize.delete_dataset(datasetArn=event_interactions_dataset_arn)
time.sleep(60)

In [ ]:
# Delete the schema
personalize.delete_schema(schemaArn=schema_arn)

In [ ]:
# Delete the Dataset Group
personalize.delete_dataset_group(datasetGroupArn=dataset_group_arn)

In [ ]:
# Empty the S3 Bucket
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
bucket.objects.all().delete()
time.sleep(60)

In [ ]:
# Delete the S3 Bucket
bucket.delete()

In [ ]:
# IAM policies should also be removed
iam = boto3.client("iam")
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/AmazonS3FullAccess", RoleName=role_name)
iam.detach_role_policy(PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess",RoleName=role_name)

iam.delete_role(RoleName=role_name)

## Último paso

Después de limpiar todos los recursos, puede cerrar esta ventana y regresar a la página de GitHub en donde comenzó. Al final del archivo Readme, encontrará los pasos que debe seguir para eliminar la pila de CloudFormation que creó anteriormente. Una vez hecho esto, habrá completado el taller por completo.

¡Felicitaciones!